# Imports

In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error
from collaborative_filtering import *
import pandas as pd
import time

# Define Data

In [2]:
item_user_matrix = np.array([
    [0,1,0,1,2,2],
    [2,3,1,1,2,2],
    [1,1,1,0,1,1],
    [0,2,3,4,1,1],
    [0,0,0,0,1,0]
])

# Recommender System with no Dim. Reduction

In [3]:
nmf_pred = nmf_wrapper(item_user_matrix)

In [4]:
mse = mean_squared_error(item_user_matrix, nmf_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.03623154133121916


# Recommender System with PCA

In [5]:
pred_nmf_pca = nmf_pca_wrapper(item_user_matrix)

In [6]:
mse = mean_squared_error(item_user_matrix, pred_nmf_pca)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.46955501445551


# Recommender System Using LU_CRTP

In [7]:
pred = lu_crtp_wrapper(item_user_matrix)

In [8]:
mse = mean_squared_error(item_user_matrix, pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.7315857487660493


# Test Loop

In [21]:
# test runtime and closeness to actual matrix
runtime_results = pd.DataFrame(columns=['n', 'NMF', 'PCA_NMF', 'LU_CRTP'])
mae_results = pd.DataFrame(columns=['n', 'NMF', 'PCA_NMF', 'LU_CRTP'])

for n in range(10000, 10001, 200):
    item_user_matrix = np.random.randint(6, size=(n, n))
    orig_matrix = np.copy(item_user_matrix)
    
    # We should be dealing with a sparse matrix, so let's delete 60% of the values
    delete_count = int(n * n * 0.6)
    delete_idx = np.unravel_index(np.random.choice(n * n, delete_count, replace=False), item_user_matrix.shape)
    item_user_matrix[delete_idx] = 0

    mae = [n]
    runtime = [n]
    for wrapper in [nmf_wrapper, nmf_pca_wrapper, lu_crtp_wrapper]:
        begin = time.time()
        mae.append(mean_squared_error(orig_matrix, wrapper(item_user_matrix)))
        runtime.append(time.time() - begin)

    print(item_user_matrix.shape)

    mae_results.loc[len(mae_results)] = mae
    runtime_results.loc[len(runtime_results)] = runtime


/home/nico/VSCodeRepos/stl-implementation/stl-implementation/.conda/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/home/nico/VSCodeRepos/stl-implementation/stl-implementation/.conda/lib/python3.11/site-packages/sklearn/decomposition/_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


(10000, 10000)


In [23]:
mae_results

,n,NMF,PCA_NMF,LU_CRTP
0,10000.0,5.166879,9.166049,5.169486
